<a href="https://colab.research.google.com/github/vohoaidanh/AIGCDetectBenchmark/blob/dire/colab/AIGCDetectbenchmark_dire_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AIGCDetectBenchmark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install ftfy -q
!pip install natsort -q
!pip install tensorboardX -q
!pip install blobfile -q
#!pip install mpi4py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 59.3 MB/s eta 0:00:00


In [ ]:
!git clone -b dire https://github.com/vohoaidanh/AIGCDetectBenchmark.git

Cloning into 'AIGCDetectBenchmark'...
remote: Enumerating objects: 681, done.
remote: Counting objects: 100% (681/681), done.
remote: Compressing objects: 100% (491/491), done.
remote: Total 681 (delta 303), reused 515 (delta 170), pack-reused 0
Receiving objects: 100% (681/681), 7.20 MiB | 13.16 MiB/s, done.
Resolving deltas: 100% (303/303), done.


In [ ]:
#!cp /content/drive/MyDrive/DATASETS/real_gen_dataset.zip -d /content
!cp /content/drive/MyDrive/DATASETS/RealFakeDB_tiny_test.zip -d /content

In [ ]:
#!unzip /content/real_gen_dataset.zip -d /content/real_gen_dataset

In [ ]:
!unzip /content/RealFakeDB_tiny_test.zip -d /content/RealFakeDB_tiny_test

In [ ]:
#!rm -r /content/real_gen_dataset/real_gen_dataset/test

In [ ]:
%cd /content/AIGCDetectBenchmark

/content/AIGCDetectBenchmark


In [ ]:
!wget https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt

--2024-03-29 15:15:36--  https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.209.18.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.209.18.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2211383297 (2.1G) [application/octet-stream]
Saving to: ‘256x256_diffusion_uncond.pt’

256x256_diffusion_u 100%[===================>]   2.06G  9.81MB/s    in 4m 0s   

2024-03-29 15:19:36 (8.80 MB/s) - ‘256x256_diffusion_uncond.pt’ saved [2211383297/2211383297]



In [ ]:
import os
os.makedirs('models', exist_ok=True)
import shutil
shutil.move('256x256_diffusion_uncond.pt', 'models')

'DIRE/models/256x256_diffusion_uncond.pt'

In [ ]:
import os
image_list = []
extensions = ['.jpg', '.png', '.gif', '.bmp', '.webp']  # Add more extensions as needed
def getname(base_dir):
    for dirpath, dirnames, filenames in os.walk(base_dir):
        if len(filenames) > 0:
            imgs = [filename for filename in filenames if any(ext in filename for ext in extensions)]
            image_list.extend(imgs)
    return image_list

image_list = getname('/content/drive/MyDrive/DATASETS/DIRE/RealFakeDB_tiny_test/dire_test')

In [ ]:
data_dir = '/content/AIGCDetectBenchmark/RealFakeDB_tiny_test'
c=0
for dirpath, dirnames, filenames in os.walk(data_dir):
    if len(filenames) >0:
        for file in filenames:
            if file in image_list:
                c+=1
                full_name = os.path.join(dirpath, file)
                os.remove(full_name)
                print('removed {}:{}:'.format(c,full_name))



In [ ]:
samples = len(os.listdir('/content/AIGCDetectBenchmark/RealFakeDB_tiny_test/test/0_real')) + len(os.listdir('/content/AIGCDetectBenchmark/RealFakeDB_tiny_test/test/1_fake'))

samples

1600

In [ ]:
!python preprocessing/DIRE/compute_dire.py \
--model_path models/256x256_diffusion_uncond.pt \
--attention_resolutions 32,16,8 \
--class_cond False \
--diffusion_steps 1000 --dropout 0.1 --image_size 256 --learn_sigma True --noise_schedule linear --num_channels 256 \
--num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm True \
--images_dir /content/AIGCDetectBenchmark/RealFakeDB_tiny_test/test \
--recons_dir /content/drive/MyDrive/DATASETS/DIRE/RealFakeDB_tiny_test/recons_test \
--dire_dir /content/drive/MyDrive/DATASETS/DIRE/RealFakeDB_tiny_test/dire_test \
--batch_size 16 --num_samples $samples  --timestep_respacing ddim20 --use_ddim True \
--has_subfolder True

Logging to /content/drive/MyDrive/DATASETS/DIRE/RealFakeDB_tiny_test/recons_test
Namespace(images_dir='/content/AIGCDetectBenchmark/RealFakeDB_tiny_test/test', recons_dir='/content/drive/MyDrive/DATASETS/DIRE/RealFakeDB_tiny_test/recons_test', dire_dir='/content/drive/MyDrive/DATASETS/DIRE/RealFakeDB_tiny_test/dire_test', clip_denoised=True, num_samples=1600, batch_size=16, use_ddim=True, model_path='models/256x256_diffusion_uncond.pt', real_step=0, continue_reverse=False, has_subfolder=True, image_size=256, num_channels=256, num_res_blocks=2, num_heads=4, num_heads_upsample=-1, num_head_channels=64, attention_resolutions='32,16,8', channel_mult='', dropout=0.1, class_cond=False, use_checkpoint=False, use_scale_shift_norm=True, resblock_updown=True, use_fp16=True, use_new_attention_order=False, learn_sigma=True, diffusion_steps=1000, noise_schedule='linear', timestep_respacing='ddim20', use_kl=False, predict_xstart=False, rescale_timesteps=False, rescale_learned_sigmas=False)
have crea